In [ ]:
%config Completer.use_jedi = False
import glob
import os
import re
import pandas
import numpy
import sqlite3

In [ ]:
dbfile="parsedata.db"

In [ ]:
inpfiles=glob.glob(r"examples\*.inp")
inpfiles

In [ ]:
inpfile=inpfiles[2]
inpfile

In [ ]:
basename,_=os.path.splitext(os.path.basename(inpfile))
basename

In [ ]:
with open(inpfile) as f:
    txt=f.read()
len(txt)

In [ ]:
for i,line in enumerate(txt.splitlines()):
    print(i,line)
    if i > 10:
        break

In [ ]:
txt=""
with open(inpfile) as f:
    for line in f:
        l = line[:80]
        l = re.sub(r"\$.*$","",re.sub(r"\$.*?\$","",l)).strip()
        if len(l) > 0:
            txt += l + "\n"
len(txt)

In [ ]:
for i,line in enumerate(txt.splitlines()):
    print(i,line)
    if i > 100:
        break

In [ ]:
for m in re.finditer("^.*?\.\.",txt,re.DOTALL):
    pass

In [ ]:
m.group()

In [ ]:
command_list=[]
arg_list=[]

for i,m1 in enumerate(re.finditer(r"\s*(.*?)\s*\.\.\s*", txt, re.DOTALL)):
        
    g1=m1.group(1)
    print("==========")
    print(i)
    print(g1)
    print("==========")
    
    m2=re.match(r"(?P<uname>.*?)\s*=\s*(?P<tcmd>.*)|(?P<icmd>.*)",g1)
    kind = m2.lastgroup
    cmd,uname=None,None
    if kind == 'icmd':
        cmd=m2.group('icmd')
        print("Command: ", cmd)
        print("Command takes no U-NAME")
    elif kind == 'tcmd':
        cmd=m2.group('tcmd')
        uname0=m2.group('uname')
        mu0=re.match(r'"(?P<quoted>.*)"|(?P<unquoted>.*)',uname0)
        uname=mu0.group(mu0.lastgroup)
        print("Command:", cmd)
        print("U-NAME: ", uname)
    if len(cmd) == 0:
        cmd = "$noop"
    command_list.append((i,cmd,uname))
    tail2=g1[m2.end():]
    
    j,m3=-1,None
    for j,m3 in enumerate(re.finditer(r'\s*(.+?)\s*=\s*(\((?:(?:\s*"[^"]+?"|\s*[^\s\)]+?)[,\s]*)+\)|(?:.*))\s*', tail2)):
        key,val=m3.group(1),m3.group(2)
        arg_list.append((i,j,key,val))
        print(f"  {j:2}",key)
        print(f"    ",val)
    tail3 = tail2[m3.end():] if j>=0 else tail2
    if len(tail3) > 0:
        arg_list.append((i,j+1,"$tail",tail3))
    print("Tail length: ",len(tail3))
    ignore_tail = (len(tail3) == 0) \
        or cmd.startswith("RUN-PERIOD") \
        or cmd.startswith("SCHEDULE")\
        or tail3.startswith("COEF ")
    
    print()
    #if i >= 90:
    if not ignore_tail:
        break

In [ ]:
command_list

In [ ]:
arg_list

In [ ]:
cl=pandas.DataFrame(command_list,columns=['i','cmd','uname']).set_index('i')
cl

In [ ]:
al=pandas.DataFrame(arg_list,columns=['i','j','key','val']).set_index(['i','j'])
al

In [ ]:
conn = sqlite3.Connection(dbfile)
try:
    cl.to_sql("command_list",conn)
    al.to_sql("arg_list",conn)
finally:
    conn.close()

In [ ]:
conn = sqlite3.Connection(dbfile)
try:
    df1=pandas.read_sql("""select "i", "cmd", "uname", "j", "key", "val" FROM "arg_extended_refg_network";""",conn)
finally:
    conn.close()
df1

In [ ]:
edges=[]
for row in df1.itertuples():
    uname2=None
    m1=re.match(r'"(?P<quoted>.*)"|(?P<unquoted>.*)',row.uname)
    uname2=m1.group(m1.lastgroup)
    
    val2=None
    m2=re.match(r'"(?P<quoted>.*)"|\(\s*(?P<list>.+?)\s*\)|(?P<unquoted>[^\(\)]*)',row.val)
    val2=m2.group(m2.lastgroup)
    
    print(f'{row.Index} "{uname2}" -> "{val2}"')
    edges.append((uname2, val2))
    #if row.Index == 15:
    #    break
el=pandas.DataFrame(edges,columns=['uname','val'])
el

In [ ]:
def clean_value(val):
    m=re.match(r'"(?P<quoted>.*)"|\(\s*(?P<list>.+?)\s*\)|(?P<unquoted>[^\(\)]*)',val)
    return m.group(m.lastgroup)

In [ ]:
df1['val'].apply(clean_value)

In [ ]:
m1

In [ ]:
m1.lastgroup

In [ ]:
m2

In [ ]:
m2.lastgroup

In [ ]:
el['uname']

In [ ]:
nodes=numpy.union1d(el.uname,el.val)
nodes

In [ ]:
nodes.shape

In [ ]:
for n in nodes:
    if n != n.strip():
        print(n)

In [ ]:
n

In [ ]:
conn = sqlite3.Connection(dbfile)
try:
    nodes2=pandas.read_sql("""select "cmd", "uname", "shape", "color" FROM "refg_nodes";""",conn)
finally:
    conn.close()
nodes2

In [ ]:
import graphviz
print(graphviz.ENGINES)
print(graphviz.FORMATS)

In [ ]:
from graphviz import Digraph

dot = Digraph(comment=basename,engine='dot',format='pdf')
# Digraph(name='pet-shop', node_attr={'shape': 'plaintext'})

for row in nodes2.itertuples():
    #dot.attr('node', shape=row.shape)
    dot.node(row.uname, shape=row.shape, fillcolor=row.color, style="filled")
for row in el.itertuples():
    dot.edge(row.uname,row.val)


In [ ]:
dot

In [ ]:
dot.format

In [ ]:
dot.render(f'output/{basename}.gv', view=True)  

In [ ]:
?pandas.read_sql

In [ ]:
def datatable_for_command(cmd):
    conn = sqlite3.Connection(dbfile)
    try:
        df2=pandas.read_sql("""select "uname", "key", "val" FROM "arg_extended" where "cmd" == ?;""",
                            conn,
                            index_col=['uname','key'],
                            params=(cmd,))
    finally:
        conn.close()
    df3=df2.unstack()
    df3.columns=df3.columns.droplevel(0)
    return df3

In [ ]:
df3=datatable_for_command("SPACE")
df3

In [ ]:
df3=datatable_for_command("PROCESS-LOAD")
df3

In [ ]:
?object

In [ ]:
df3.astype(df3.dtypes)

In [ ]:
dt=df3.dtypes
for c in df3:
    print(c)
    try:
        df3[c].astype(float)
        dt[c] = 'float'
        print("OK to float")
    except:
        print("Not a float!")
dt

In [ ]:
df3=df3.astype(dt)
df3

In [ ]:
df3.to_excel(f"output/{basename}.xlsx",sheet_name="PROCESS-LOAD",freeze_panes=(1,1))

In [ ]:
df3.to_clipboard()

In [ ]:
# http://www.graphviz.org/doc/info/shapes.html#html
# let codes=$("body > div > div > div > main > div > figure code")
# codes.map(function(x,i){return i.innerText;})

shapeopts=["box", "polygon", "ellipse", "oval", "circle", "point", "egg", "triangle", "plaintext", "plain", "diamond",
 "trapezium", "parallelogram", "house", "pentagon", "hexagon", "septagon", "octagon", "doublecircle",
 "doubleoctagon", "tripleoctagon", "invtriangle", "invtrapezium", "invhouse", "Mdiamond", "Msquare",
 "Mcircle", "rect", "rectangle", "square", "star", "none", "underline", "cylinder", "note", "tab",
 "folder", "box3d", "component", "promoter", "cds", "terminator", "utr", "primersite", "restrictionsite",
 "fivepoverhang", "threepoverhang", "noverhang", "assembly", "signature", "insulator", "ribosite",
 "rnastab", "proteasesite", "proteinstab", "rpromoter", "rarrow", "larrow", "lpromoter"]
len(shapeopts)

In [ ]:
?df3.to_excel

In [ ]:
?conn.executescript